# filter.ipynb

추가 이미지 중 테스트 이미지와 중복되는 이미지 제거하는 스크립트

### 라이브러리

In [2]:
import os
import json
import shutil
from torchvision import transforms
from PIL import Image
import torch
import hashlib
import pandas as pd

### 경로

In [4]:
add_train_img_dir = '../data/additional_data/images/train'
add_val_img_dir = '../data/additional_data/images/val'

add_train_label_dir = '../data/additional_data/labels/train'
add_val_label_dir = '../data/additional_data/labels/val'

duplicated_img_dir = '../data/additional_data/duplicated_images'
duplicated_label_dir = '../data/additional_data/duplicated_labels'

test_img_dir = '../data/ai02-level1-project/test_images'

## 이미지 검출

### 이미지 경로 읽어서 해시값 생성

In [5]:
def image_hash(path):
    """이미지 파일을 읽어서 md5 해시값 생성"""
    with open(path, "rb") as f:
        return hashlib.md5(f.read()).hexdigest()

### 학습 이미지 해시값 생성

In [6]:
train_hashes = {}
for fname in os.listdir(add_train_img_dir):
    fpath = os.path.join(add_train_img_dir, fname)
    h = image_hash(fpath)
    train_hashes[h] = fname

### 검증 이미지 해시값 생성

In [21]:
val_hashes = {}
for fname in os.listdir(add_val_img_dir):
    fpath = os.path.join(add_val_img_dir, fname)
    h = image_hash(fpath)
    val_hashes[h] = fname

### 테스트 이미지에서 중복 탐색

In [22]:
trina_duplicates = []
val_duplicates = []

for fname in os.listdir(test_img_dir):
    fpath = os.path.join(test_img_dir, fname)
    h = image_hash(fpath)
    if h in train_hashes.keys():
        trina_duplicates.append((fname, train_hashes[h]))
    elif h in val_hashes.keys():
        val_duplicates.append((fname, val_hashes[h]))

In [23]:
print(f"train 중복 이미지 수: {len(trina_duplicates)}")
print(f"val 중복 이미지 수: {len(val_duplicates)}")

train 중복 이미지 수: 832
val 중복 이미지 수: 0


### trian 이미지 중 중복된 이미지 및 labels 옮기기

In [ ]:
for h, img_name in trina_duplicates:
    label_name = img_name.replace('.png', '.txt')
    old_img_path = os.path.join(add_train_img_dir, img_name)
    new_img_path = os.path.join(duplicated_img_dir, img_name)
    shutil.move(old_img_path, new_img_path)

    old_label_path = os.path.join(add_train_label_dir, label_name)
    new_label_path = os.path.join(duplicated_label_dir, label_name)
    shutil.move(old_label_path, new_label_path)